# Vectorisation 
## X-vectors

pip install speechbrain

In [1]:
# librairies 
import glob 
import speechbrain as sb
import pandas as pd
import torch
import os
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
import time
import logging

/home/lise/Documents/TAL_M2S3/CNN/venv_CNN/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [2]:
# Démarrer le timer
start_time = time.time()

In [5]:
# Ouverture fichiers

chemin_base = "/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec" 

list_fichier = glob.glob(f"{chemin_base}/ESLO2_*/*_16000Hz.wav")
print(list_fichier[:10])

# On récupère le csv contenant les infos sur les locuteurs : 
metadata_fichier = pd.read_csv("./data/metadonnees_ESLO2_ENT_ENTJEUN.csv")

['./data/ESLO2_ENT_1001/ESLO2_ENT_1001_0_16000Hz.wav', './data/ESLO2_ENT_1001/ESLO2_ENT_1001_1_16000Hz.wav', './data/ESLO2_ENT_1001/ESLO2_ENT_1001_2_16000Hz.wav']


In [6]:
print("Nombre de fichiers audios :", len(list_fichier))

Nombre de fichiers audios : 3


In [7]:
# rajout classses d'age
def class_age(age):
	if age == 'vieux' or int(age) > 60: 
		return "vieux"
	elif int(age) < 30:
		return "jeune"
	else:
		return "mid"
	
metadata_fichier['classe_age'] = metadata_fichier['age'].apply(class_age)

In [8]:
# vectorisation 
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

def vectorisation(fichier):
	signal, fs = torchaudio.load(fichier)
	embeddings = classifier.encode_batch(signal)
	return embeddings


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
/home/lise/Documents/TAL_M2S3/CNN/venv_CNN/lib/python3.12/site-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
I

____________________

### Ne pas lancer !!

In [9]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION GIT 

for fichier in list_fichier: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if not os.path.exists(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
		os.makedirs(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
	chemin = f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
	
	torch.save(embeddings, chemin)


In [ ]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION uniquement HOMME sur GIT  

for fichier in list_fichier: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if sexe == "homme":
		if not os.path.exists(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
			os.makedirs(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
		chemin = f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
		
		torch.save(embeddings, chemin)
	else: 
		continue


In [27]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION DISQUE DUR  

base_disque = "/media/lise/SMARTDISK/BrissetLise_M2TAL_age"

for fichier in list_fichier[:5]: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if not os.path.exists(f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
		os.makedirs(f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
	chemin = f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
	
	torch.save(embeddings, chemin)
	print(f"Enregistrement du fichier {fichier} dans {chemin}")

RuntimeError: Couldn't find appropriate backend to handle uri /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_0_16000Hz.wav and format None.

______________

### Lancer à partir d'ici :

In [21]:
# Configuration du logging
logging.basicConfig(filename='process.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [26]:
##### CETTE VERSION EST LA BONNE #####

base_disque = "/media/lise/SMARTDISK/BrissetLise_M2TAL_age"

for fichier in list_fichier[:10]: 
    print(fichier)
    try:
        embeddings = vectorisation(fichier)
        directory_name = fichier.split("/")[-2]

        # Vérification de l'existence du répertoire dans les métadonnées
        if directory_name in metadata_fichier['directory'].values:
            age = metadata_fichier[metadata_fichier['directory'] == directory_name]['classe_age'].values[0]
            sexe = metadata_fichier[metadata_fichier['directory'] == directory_name]['sexe'].values[0]
            logging.info(f"Traitement du fichier: {fichier}, Age: {age}, Sexe: {sexe}")

            # Création du répertoire si nécessaire
            target_directory = f".{base_disque}/xvectors_4sec/{sexe}/{age}/{directory_name}"
            if not os.path.exists(target_directory):
                os.makedirs(target_directory)
                logging.info(f"Création du répertoire: {target_directory}")

            chemin = f"{target_directory}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
            
            # Sauvegarde des embeddings
            torch.save(embeddings, chemin)
            logging.info(f"Embeddings sauvegardés dans: {chemin}")
        else:
            logging.warning(f"Répertoire non trouvé dans les métadonnées: {directory_name}")

    except Exception as e:
        logging.error(f"Erreur lors du traitement du fichier {fichier}: {e}")




# Arrêter le timer
end_time = time.time()

# Calculer le temps écoulé
elapsed_time = end_time - start_time
print(f"Temps d'exécution du programme : {elapsed_time} secondes")

ERROR:root:Erreur lors du traitement du fichier /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_0_16000Hz.wav: Couldn't find appropriate backend to handle uri /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_0_16000Hz.wav and format None.
ERROR:root:Erreur lors du traitement du fichier /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_1_16000Hz.wav: Couldn't find appropriate backend to handle uri /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_1_16000Hz.wav and format None.
ERROR:root:Erreur lors du traitement du fichier /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_2_16000Hz.wav: Couldn't find appropriate backend to handle uri /media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_2_16000Hz.w

/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_0_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_1_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_2_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_3_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_4_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_5_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_6_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_1221_7_16000Hz.wav
/media/lise/SMARTDISK/DufourOrane_Memoire-M2TAL/Corpus4Sec/ESLO_4Sec/ESLO2_DIA_1221/ESLO2_DIA_12